In [9]:
import os

os.chdir("/home/code/PaddleOCR")

In [34]:
from pathlib import Path
from ppocr.utils.korean_compose import decompose_korean_char
import tqdm

path = Path("/home/datasets/aihub_rec/horizontal_label.txt")
target_path = Path("/home/datasets/aihub_rec/horizontal_label2.txt")
target_path.unlink()
with open(path) as f:
    lines = [line.rstrip("\n").split("\t") for line in f.readlines()]

with open(target_path, "a+") as f:
    for path, label in tqdm.tqdm(lines):
        decomposed = decompose_korean_char(label)
        first = "".join([x[0] for x in decomposed])
        second = "".join([x[1] for x in decomposed])
        third = "".join([x[2] for x in decomposed])
        dict_label = {
            "label":label,
            "first":first,
            "second":second,
            "third":third,
        }
        f.write(f"""{path}\t{str(dict_label).replace("'", '"')}\n""")    

100%|██████████| 682525/682525 [00:10<00:00, 67578.68it/s]


In [24]:
a = "안녕"
b = "그래"
c = "나도"
list(zip(a, b, c))

[('안', '그', '나'), ('녕', '래', '도')]

In [21]:
import functools


first_grapheme_dict = {
            'ᄀ': '가', 'ᄁ': '까', 'ᄂ': '나', 'ᄃ': '다', 'ᄄ': '따', 'ᄅ': '라',
            'ᄆ': '마', 'ᄇ': '바', 'ᄈ': '빠', 'ᄉ': '사', 'ᄊ': '싸', 'ᄋ': '아',
            'ᄌ': '자', 'ᄍ': '짜', 'ᄎ': '차', 'ᄏ': '카', 'ᄐ': '타', 'ᄑ': '파', 'ᄒ': '하'
        }


def extract_first_grapheme(text, type="first"):
    result = ''
    for char in text:
        # 한글 유니코드 범위인지 확인
        if 44032 <= ord(char) <= 55203:
            # 초성 구하기
            choseong_index = (ord(char) - 44032) // 588
            # 초성을 문자로 변환하여 결과에 추가
            choseong = chr(choseong_index + 0x1100)  # 초성 유니코드 시작값은 0x1100입니다.
            result += first_grapheme_dict[choseong]
        else: # 한글이 아니면 그대로 결과에 추가
            result += char
            
    return result

sceond_grapheme_dict = {"ᅡ":"아","ᅢ":"애","ᅣ":"야","ᅤ":"얘","ᅥ":"어","ᅦ":"에","ᅧ":"여",
"ᅨ":"예","ᅩ":"오","ᅪ":"와","ᅫ":"왜","ᅬ":"외","ᅭ":"요","ᅮ":"우",
"ᅯ":"워","ᅰ":"웨","ᅱ":"위","ᅲ":"유","ᅳ":"으","ᅴ":"의","ᅵ":"이"}


def extract_second_grapheme(text):
    result = ''
    for char in text:
        # 한글 유니코드 범위인지 확인
        if 44032 <= ord(char) <= 55203:
            # 중성 구하기
            jungseong_index = ((ord(char) - 44032) % 588) // 28
            # 중성을 문자로 변환하여 결과에 추가
            jungseong = chr(jungseong_index + 0x1161)  # 중성 유니코드 시작값은 0x1161입니다.
            result += sceond_grapheme_dict[jungseong]
        else:
            # 한글이 아니면 그대로 결과에 추가
            result += char
        
    return result



third_grapheme_dict = {"ᆨ":"윽","ᆫ":"은","ᆮ":"읃","ᆯ":"을","ᆷ":"음","ᆸ":"읍","ᆺ":"읏",
"ᆼ":"응","ᆽ":"읒","ᆾ":"읓","ᆿ":"읔","ᇀ":"읕","ᇁ":"읖","ᇂ":"읗",
"ᆩ":"윾","ᆻ":"읐","ᆪ":"윿","ᆬ":"읁","ᆭ":"읂","ᆰ":"읅","ᆱ":"읆",
"ᆲ":"읇","ᆳ":"읈","ᆴ":"읉","ᆵ":"읊","ᆶ":"읋","ᆹ":"읎"}


def extract_third_grapheme(text):
    result = ''
    has_jongseong = False  # 받침이 있는지 여부를 판별하기 위한 변수

    for char in text:
        # 한글 유니코드 범위인지 확인
        if 44032 <= ord(char) <= 55203:
            # 종성 구하기
            jongseong_index = (ord(char) - 44032) % 28

            # 받침이 있는 경우에만 결과에 추가하고, 받침이 없는 경우 "으"를 추가
            if jongseong_index != 0:
                jongseong = chr(jongseong_index + 0x11A7)  # 종성 유니코드 시작값은 0x11A8이 아닌 0x11A7입니다.
                result += third_grapheme_dict[jongseong]
            else:
                result += '으'
        else:
            # 한글이 아니면 그대로 결과에 추가
            result += char
    
    return result

@functools.lru_cache(maxsize=1000000)
def extract_grapheme(text):
    return {
        "first":extract_first_grapheme(text),
        "second":extract_second_grapheme(text),
        "third":extract_third_grapheme(text)
    }

extract_grapheme("한명훈 않아ㄲㅆㅉ")


{'first': '하마하 아아ㄲㅆㅉ', 'second': '아여우 아아ㄲㅆㅉ', 'third': '은응은 읂으ㄲㅆㅉ'}

In [42]:
grapheme_to_class = {
    "first" : {'ᄀ': '가', 'ᄁ': '까', 'ᄂ': '나', 'ᄃ': '다', 'ᄄ': '따', 'ᄅ': '라',
                'ᄆ': '마', 'ᄇ': '바', 'ᄈ': '빠', 'ᄉ': '사', 'ᄊ': '싸', 'ᄋ': '아',
                'ᄌ': '자', 'ᄍ': '짜', 'ᄎ': '차', 'ᄏ': '카', 'ᄐ': '타', 'ᄑ': '파', 'ᄒ': '하'},
    "second": {"ᅡ":"아","ᅢ":"애","ᅣ":"야","ᅤ":"얘","ᅥ":"어","ᅦ":"에","ᅧ":"여",
                "ᅨ":"예","ᅩ":"오","ᅪ":"와","ᅫ":"왜","ᅬ":"외","ᅭ":"요","ᅮ":"우",
                "ᅯ":"워","ᅰ":"웨","ᅱ":"위","ᅲ":"유","ᅳ":"으","ᅴ":"의","ᅵ":"이"},
    "third" : {"ᆨ":"윽","ᆫ":"은","ᆮ":"읃","ᆯ":"을","ᆷ":"음","ᆸ":"읍","ᆺ":"읏",
                "ᆼ":"응","ᆽ":"읒","ᆾ":"읓","ᆿ":"읔","ᇀ":"읕","ᇁ":"읖","ᇂ":"읗",
                "ᆩ":"윾","ᆻ":"읐","ᆪ":"윿","ᆬ":"읁","ᆭ":"읂","ᆰ":"읅","ᆱ":"읆",
                "ᆲ":"읇","ᆳ":"읈","ᆴ":"읉","ᆵ":"읊","ᆶ":"읋","ᆹ":"읎"}   
    }
initial_list = [
    'ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ',
    'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ'
]
# 중성 리스트
medial_list = [
    'ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 
    'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ'
]
# 종성 리스트
final_list = [
    '', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 
    'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 
    'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ'
]


class_to_grapheme = {k: {_v:_k for _k, _v in v.items()} for k, v in grapheme_to_class.items()}

def f(pred):
    for grapheme in ["first", "second", "third"]:
        pred[grapheme] = [class_to_grapheme[grapheme].get(x, x) for x in pred[grapheme]]
    return list(zip(*pred.values()))
    return [compose_korean_char(x, y, z) for x, y, z in zip(*pred.values())]
        
    
    
    return pred


x = {'first': '하마하 아아ㄲㅆㅉ', 'second': '아여우 아아ㄲㅆㅉ', 'third': '은응은 읂으ㄲㅆㅉ'}
f(x)

[('ᄒ', 'ᅡ', 'ᆫ'),
 ('ᄆ', 'ᅧ', 'ᆼ'),
 ('ᄒ', 'ᅮ', 'ᆫ'),
 (' ', ' ', ' '),
 ('ᄋ', 'ᅡ', 'ᆭ'),
 ('ᄋ', 'ᅡ', '으'),
 ('ㄲ', 'ㄲ', 'ㄲ'),
 ('ㅆ', 'ㅆ', 'ㅆ'),
 ('ㅉ', 'ㅉ', 'ㅉ')]

In [5]:
class_to_grapheme

{'first': {'가': 'ᄀ',
  '까': 'ᄁ',
  '나': 'ᄂ',
  '다': 'ᄃ',
  '따': 'ᄄ',
  '라': 'ᄅ',
  '마': 'ᄆ',
  '바': 'ᄇ',
  '빠': 'ᄈ',
  '사': 'ᄉ',
  '싸': 'ᄊ',
  '아': 'ᄋ',
  '자': 'ᄌ',
  '짜': 'ᄍ',
  '차': 'ᄎ',
  '카': 'ᄏ',
  '타': 'ᄐ',
  '파': 'ᄑ',
  '하': 'ᄒ'},
 'second': {'아': 'ᅡ',
  '애': 'ᅢ',
  '야': 'ᅣ',
  '얘': 'ᅤ',
  '어': 'ᅥ',
  '에': 'ᅦ',
  '여': 'ᅧ',
  '예': 'ᅨ',
  '오': 'ᅩ',
  '와': 'ᅪ',
  '왜': 'ᅫ',
  '외': 'ᅬ',
  '요': 'ᅭ',
  '우': 'ᅮ',
  '워': 'ᅯ',
  '웨': 'ᅰ',
  '위': 'ᅱ',
  '유': 'ᅲ',
  '으': 'ᅳ',
  '의': 'ᅴ',
  '이': 'ᅵ'},
 'third': {'윽': 'ᆨ',
  '은': 'ᆫ',
  '읃': 'ᆮ',
  '을': 'ᆯ',
  '음': 'ᆷ',
  '읍': 'ᆸ',
  '읏': 'ᆺ',
  '응': 'ᆼ',
  '읒': 'ᆽ',
  '읓': 'ᆾ',
  '읔': 'ᆿ',
  '읕': 'ᇀ',
  '읖': 'ᇁ',
  '읗': 'ᇂ',
  '윾': 'ᆩ',
  '읐': 'ᆻ',
  '윿': 'ᆪ',
  '읁': 'ᆬ',
  '읂': 'ᆭ',
  '읅': 'ᆰ',
  '읆': 'ᆱ',
  '읇': 'ᆲ',
  '읈': 'ᆳ',
  '읉': 'ᆴ',
  '읊': 'ᆵ',
  '읋': 'ᆶ',
  '읎': 'ᆹ'}}